# Homework 2: Monte Carlo simulation 
Using a Monte Carlo simulation approach, we will model variability in an industrial system filling containers with a fluid. Let’s say there is a factory filling 10,000 containers (150 mL each) per day with ethylene glycol. To fill the containers, nozzles open to release ethylene glycol from a reservoir such that the volume released into a 150 mL container is modeled by:

Volume = time the nozzle is open x nozzle area x fluid viscosity

Or more mathematically:
$ V = t*A*32.7e^{\frac{T}{320}} $

Where:

$t$ = the time the nozzle is open (seconds)

$A$ = nozzle area (cm^2)

$T$ = room temperature (celsius)

### 1. Using this equation, set t=3.45 seconds, T= 21°C, and A=1.2 cm^2 and calculate the deterministic (expected) volume (mL of ethylene glycol) that will be released into each 150 mL container.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# The function is designed to process and decide how many repetitions of volume calculation are needed
# This function then creates a time variable with the tolerance allowed in the prompt
# After the function either returns all the data as an array, or just the one value

def volume_finder(repetitions):
   
    if repetitions == 1:
        time_tolerance = np.random.randint(-5, 5) / 100
        time_passed = 3.45 # seconds
        time = time_passed + time_tolerance
        temperature = 21 # Celsius
        area = 1.2 # cm^2
        viscosity = 32.7 * np.exp( ( temperature / 320 ) )

        volume = time * area * viscosity
        volume = np.round(volume, 2)
        
        return volume 
    
    elif repetitions > 1:
        volume_data = []
        for i in range(repetitions):
            time_tolerance = np.random.randint(-5, 5) / 100
            time_passed = 3.45 # seconds
            time = time_passed + time_tolerance
            temperature = 21 # Celsius
            area = 1.2 # cm^2
            viscosity = 32.7 * np.exp( ( temperature / 320 ) )

            volume =  time * area * viscosity
            volume = np.round(volume, 2)
            
            volume_data.append(volume)
        return volume_data
    
    else:
        print('The input was not able to be processed.')
        return None 


# how_many = 5
# print(volume_finder(how_many))

### 2. The nozzle manufacturer says there is a timing tolerance of up to ±0.05 seconds, which may be longer with old or well-used equipment. Using the np.random.randn(), simulate this temporal uncertainty to the timing variable, t, and calculate one potential volume outcome.

### 3.  Repeat the calculations for 10,000 containers and make a histogram of the outcomes.

Error: invalid literal for int() with base 10: 'a'
Error: invalid literal for int() with base 10: 'b'
Error: invalid literal for int() with base 10: ''
Error: invalid literal for int() with base 10: 'c'
Error: invalid literal for int() with base 10: 'd'
Error: invalid literal for int() with base 10: 'e'
Error: invalid literal for int() with base 10: 'f'
Error: invalid literal for int() with base 10: 'g'
Error: invalid literal for int() with base 10: 'exit'
Error: invalid literal for int() with base 10: "'exit"
Error: invalid literal for int() with base 10: "'exit'"


### 3a. Calculate the mean, standard deviation, and range of the 10,000 outcomes. Based on these findings, would you trust purchasing a container from this factory? Support your answer.

### 3b. Using the results from 3a, calculate the cumulative probability of releasing a volume exceeding (i.e. overfilling) the 150 mL container. Does that level of overfilled containers seem acceptable for a profitable business?

### 4. Engineering tolerances aren’t the only factor to consider in complex systems. In this case, fluid viscosity (and thus the volume released in a given time) is affected by temperature. It is common for a factory room to heat up by several degrees due to motorized equipment running for multiple hours. In addition to the nozzle timing uncertainty, add a thermal uncertainty of ±4°C to the room temperature variable, T.


### 4a. Calculate the volumes for 10,000 outcomes and make a histogram.

### 4b. Calculate the mean, standard deviation, and range of the 10,000 outcomes. Based on these findings, would you change your opinion from 3a? Support your answer.

## 4c. Using the results from 4b, calculate the cumulative probability of overfilling a 150 mL container.

### 4d. Calculate the cumulative probability of UNDERfilling a container by 5% of the deterministic volume from question 1.

### 4e. Calculate the cumulative probability of filling a container within 1mL of the deterministic volume. 

### Extra Credit: How many containers would be overfilled with both the temporal and thermal uncertainty conditions? If each overfilled container cost the company 1.25USD in lost product and cleaning time and it would cost 250kUSD to insulate the equipment. Would it be profitable to insulate the equipment over one year? Over another time period? Recall, the company produces 10k containers per day.